In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import json
from IPython.display import display

In [2]:
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer

In [114]:
df = pd.read_csv('output_locations_temp.csv', usecols = ['id', 'cities', 'countries'] )
df['cities'] = df['cities'].str.replace(',', ' ').str.strip("[]").str.replace("'", ' ')
df['cities'] = df.cities.str.split()
df = df.dropna(subset=['cities', 'countries'], how="all")


locations = pd.read_csv('https://raw.githubusercontent.com/datasets/world-cities/master/data/world-cities.csv')


mapping = locations.iloc[:,0:2].groupby('country', as_index=False, sort = True).agg(', '.join)
mapping['name'] = mapping.name.apply(lambda x: x.split(', '))
map_dict = dict(zip(mapping.country, mapping.name))

# for x in df['cities']: 
#     if type(x) is list and len(x) <2:
#         for y in x:
#             for k,v in map_dict.items():
#                 if y in v:
#                     print( k, y)


In [55]:
# JUST CLEANING SHIT

dropped = df[pd.notnull(df['countries'])]
popdf = pd.read_csv('engagement_output.csv')
merged = dropped.merge(popdf, on="id")
merged = merged[['countries','score']]
merged['countries'] = merged['countries'].str.replace(',', ' ').str.strip("[]").str.replace("'", ' ')
merged['countries'] = merged.countries.str.split()

display(merged)



countries     score
0                     [Turkey]  0.100000
1                     [Cyprus]  0.226796
2                      [Italy]  0.166187
3                     [Canada]  0.100000
4     [Turkmenistan, Malaysia]  0.132322
...                        ...       ...
2756                   [Italy]  0.114958
2757                [Portugal]  0.123708
2758                   [Japan]  0.296330
2759                [Portugal]  0.100000
2760                [Portugal]  0.129917

[2761 rows x 2 columns]

In [5]:
def splitter(df,target_column):
    ''' df = dataframe to split,
    target_column = the column containing the values to split
    separator = the symbol used to perform the split
    returns: a dataframe with each entry for the target column separated, with each element moved into a new row. 
    The values in the other columns are duplicated across the newly divided rows.
    '''
    def splitListToRows(row,row_accumulator,target_column):
        split_row = row[target_column]
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column))
    new_df = pd.DataFrame(new_rows)
    return new_df

seperate = splitter(merged, 'countries')

In [6]:
sep_df = seperate.groupby('countries')['score'].sum()

sep_df = pd.DataFrame(sep_df)
sep_df = sep_df.reset_index()
sep_df.loc[137] = ['United States of America', 52.521498]
sep_df.loc[138] = ['South Korea', 8.321705]
sep_df.loc[139] = ['United Kingdom', 14.032491]
sep_df.loc[140] = ['Costa Rica', 0.216953]

sep_df.fillna('No data', inplace = True)
display(sep_df.sort_values(by=['score'], ascending=False))


countries      score
137  United States of America  52.521498
56                      India  27.138134
61                      Italy  21.532087
128                    Turkey  17.952119
24                     Canada  16.613434
..                        ...        ...
1                     Algeria   0.114958
119                  Suriname   0.114958
42                     Gambia   0.114958
87                   Mongolia   0.100000
2                      Angola   0.100000

[141 rows x 2 columns]

In [7]:
import pandas as pd
import geopandas as gpd

shapefile = 'countries_data/ne_110m_admin_0_countries.shp'
gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]
gdf.columns = ['country', 'country_code', 'geometry']
gdf = gdf.drop(gdf.index[159])
gdf.head()

merged = gdf.merge(sep_df, left_on='country', right_on='countries',  how = 'left')



display(merged[merged['country'] == 'India' ])
southkorea = merged[merged['country'] == 'South Korea']
usa = merged[merged['country'] == 'United States of America']

display(southkorea)
display(usa)


country country_code                                           geometry  \
98   India          IND  POLYGON ((97.32711 28.26158, 97.40256 27.88254...   

   countries      score  
98     India  27.138134

country country_code  \
96  South Korea          KOR   

                                             geometry    countries     score  
96  POLYGON ((126.17476 37.74969, 126.23734 37.840...  South Korea  8.321705

country country_code  \
4  United States of America          USA   

                                            geometry  \
4  MULTIPOLYGON (((-122.84000 49.00000, -120.0000...   

                  countries      score  
4  United States of America  52.521498

In [8]:
import json
#Read data to json.
merged_json = json.loads(merged.to_json())
#Convert to String like object.
json_data = json.dumps(merged_json)

In [13]:
from bokeh.io import curdoc, output_notebook
from bokeh.models import Slider, HoverTool
from bokeh.layouts import widgetbox, row, column


#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)

#Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][8]

#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors. Input nan_color.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 40, nan_color = '#d9d9d9')

#Define custom tick labels for color bar.
tick_labels = {'0': '0', '5': '5', '10':'10', '15':'15', '20':'20', '25':'25', '30':'30', '35':'35','40':'<40'}

#Add hover tool
hover = HoverTool(tooltips = [ ('Country','@country'),('Popularity', '@score')])


#Create color bar.
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
                     border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)

#Create figure object.
p = figure(title = 'Most Talked About Destinations Leading Up to Christmas', plot_height = 600 , plot_width = 950, toolbar_location = None, tools = [hover])
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'score', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)


p.add_layout(color_bar, 'below')

# Define the callback function: update_plot
def update_plot(attr, old, new):
    yr = slider.value
    new_data = json_data(yr)
    geosource.geojson = new_data
    p.title.text = 'Share of adults who are obese, %d' %yr
    

# Make a column layout of widgetbox(slider) and plot, and add it to the current document
layout = column(p)
curdoc().add_root(layout)

#Display plot inline in Jupyter notebook
output_notebook()

#Display plot
show(layout)

Loading BokehJS ...